<a href="https://colab.research.google.com/github/BlueBerry-Coder/Practice/blob/main/dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
from PIL import Image
from torch.utils.data import Dataset
import numpy as np
from glob import glob
import matplotlib.pylab as plt
import pydicom as dicom
import nibabel as nib
from sklearn.preprocessing import MinMaxScaler
from skimage.transform import rotate

In [ ]:
TRAINING_DATA_PATH = '../input/brats20-dataset-training-validation/BraTS2020_TrainingData/MICCAI_BraTS2020_TrainingData/'
VALIDATION_DATA_PATH = '../input/brats20-dataset-training-validation/BraTS2020_ValidationData/MICCAI_BraTS2020_ValidationData'

    # Training Data,
        #t1-исходные, t1ce-постконтрастные взвешенные, t2-взвешенные, flair-с подавленной инверсией восстановлением жидкости, seg-сегментированные

train_flair_dir = nib.load(TRAINING_DATA_PATH + 'BraTS20_Training_001/BraTS20_Training_001_flair.nii').get_fdata()
train_mask_dir = nib.load(TRAINING_DATA_PATH + 'BraTS20_Training_001/BraTS20_Training_001_seg.nii').get_fdata()
train_t1_dir = nib.load(TRAINING_DATA_PATH + 'BraTS20_Training_001/BraTS20_Training_001_t1.nii').get_fdata()
train_t1ce_dir = nib.load(TRAINING_DATA_PATH + 'BraTS20_Training_001/BraTS20_Training_001_t1ce.nii').get_fdata()
train_t2_dir = nib.load(TRAINING_DATA_PATH + 'BraTS20_Training_001/BraTS20_Training_001_t2.nii').get_fdata()

#path = '../input/brats20-dataset-training-validation/BraTS2020_TrainingData/MICCAI_BraTS2020_TrainingData/BraTS20_Training_001/BraTS20_Training_001_t1.nii'

    # Validation Data
        #t1-исходные, t1ce-постконтрастные взвешенные, t2-взвешенные, flair-с подавленной инверсией восстановлением жидкости

val_flair_dir = nib.load(TRAINING_DATA_PATH + 'BraTS20_Validation_001/BraTS20_Validation_001_flair.nii').get_fdata()
val_t1_dir = nib.load(TRAINING_DATA_PATH + 'BraTS20_Validation_001/BraTS20_Validation_001_t1.nii').get_fdata()
val_t1ce_dir = nib.load(TRAINING_DATA_PATH + 'BraTS20_Validation_001/BraTS20_Validation_001_t1ce.nii').get_fdata()
val_t2_dir = nib.load(TRAINING_DATA_PATH + 'BraTS20_Validation_001/BraTS20_Validation_001_t2.nii').get_fdata()

#path = '../input/brats20-dataset-training-validation/BraTS2020_TrainingData/MICCAI_BraTS2020_ValidationData/BraTS20_Validation_001/BraTS20_Validation_001_t1.nii'

In [ ]:
class BratsDataset(Dataset):
    def __init__(self, train_t1_dir, train_mask_dir, transform=None):
        self.train_t1_dir = train_t1_dir
        self.train_mask_dir = train_mask_dir
        self.transform = transform
        self.images = os.listdir(train_t1_dir)

    def __len__(self):
        return len(self.images)

    def __getitem__(self, index):
        img_path = os.path.join(self.train_t1_dir, self.images[index])
        mask_path = os.path.join(self.train_mask_dir, self.images[index].replace(".jpg", "_mask.gif"))
        image = np.array(Image.open(img_path).convert("RGB"))
        mask = np.array(Image.open(mask_path).convert("L"), dtype=np.float32)
        mask[mask == 255.0] = 1.0

        if self.transform is not None:
            augmentations = self.transform(image=image, mask=mask)
            image = augmentations["image"]
            mask = augmentations["mask"]

        return image, mask

In [ ]:
# исследование данных
img = nib.load(path).get_fdata()
img.shape

plt.style.use('default')
fig, axes = plt.subplots(13,10, figsize=(120,120))
for i, ax in enumerate(axes.reshape(-1)):
    ax.imshow(img[:, :, 1 + i])
plt.show()